## Combine the data

In [ ]:
# get the code
import sys
sys.path.append('../code')
from combineCNVdata import get_covNsnp
from rollingCNV import add_rolling_data

In [ ]:
# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'
# standard paths
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
static_path = os.path.join(home, "Dropbox/Icke/Work/static")
cluster_path = os.path.join(home, "/Users/martinscience/mount")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")

# tool-specific paths
shell_path = "../shell"
# 
cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")

# the path to the input data
cnv_path = os.path.join(cnvdata, "cnv")
cnvPON_path = cnv_path = os.path.join(cnvdata, "chromCov")
cnv_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/cnv")
cnvPON_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/chromCov")

In [ ]:
cnvPON_path

In [ ]:
sample = "01_A"
snp_df, cov_df = get_covNsnp(sample, sample_cnv_path=cnv_path, PON_cnv_path=cnvPON_path, verbose=False, centerSNP=True)

In [ ]:
config = {
    'heteroSNP': {
        'VAFlimits':[0.05,0.95],
        'normalize': True,
        'windows': {
            'absVAF': {
                'sum': 20
            },
            'VAF': {
                'std': 20
            },
            'deltaVAF': {
                'std': 20
            }
        },
        'minEBscore':0.5,
        'minDepth': 30,
    },
    'coverage': {
        'VAFlimits':[0.05,0.95],
        'normalize':False,
        'windows': {
            'log2ratio': {
                'mean':500
            }
        }
    }
}

snpR_df, covR_df = add_rolling_data(snp_df, cov_df, config)

In [ ]:
snpR_df[:10]

In [ ]:
covR_df[:10]

In [ ]:
snpR_df.to_csv(os.path.join(output_path, 'heteroSNP/01_A.snp'), sep='\t', index=False)
covR_df.to_csv(os.path.join(output_path, 'covDif/01_A.cov'), sep='\t', index=False)